# COMP 562 – Lecture 11  
$$
\renewcommand{\xx}{\mathbf{x}}
\renewcommand{\yy}{\mathbf{y}}
\renewcommand{\zz}{\mathbf{z}}
\renewcommand{\vv}{\mathbf{v}}
\renewcommand{\bbeta}{\boldsymbol{\mathbf{\beta}}}
\renewcommand{\mmu}{\boldsymbol{\mathbf{\mu}}}
\renewcommand{\ssigma}{\boldsymbol{\mathbf{\sigma}}}
\renewcommand{\reals}{\mathbb{R}}
\renewcommand{\loglik}{\mathcal{LL}}
\renewcommand{\penloglik}{\mathcal{PLL}}
\renewcommand{\likelihood}{\mathcal{L}}
\renewcommand{\Data}{\textrm{Data}}
\renewcommand{\given}{ \big| }
\renewcommand{\MLE}{\textrm{MLE}}
\renewcommand{\EE}{\mathbb{E}}
\renewcommand{\KL}{\textrm{KL}}
\renewcommand{\Bound}{\mathcal{B}}
\renewcommand{\tth}{\textrm{th}}
\renewcommand{\Gaussian}[2]{\mathcal{N}\left(#1,#2\right)}
\renewcommand{\norm}[1]{\left\lVert#1\right\rVert}
\renewcommand{\ones}{\mathbf{1}}
\renewcommand{\diag}[1]{\textrm{diag}\left( #1 \right)}
\renewcommand{\sigmoid}[1]{\sigma\left(#1\right)}
\renewcommand{\myexp}[1]{\exp\left\{#1\right\}}
\renewcommand{\mylog}[1]{\log\left\{#1\right\}}
\renewcommand{\argmax}{\mathop{\textrm{argmax}}}
\renewcommand{\new}{\textrm{new}}
\renewcommand{\old}{\textrm{old}}
$$

# Examples of Unsupervised Learning

* Dimensionality reduction -- lossy compression 
* Clustering -- assigning each point to a representative cluster
  * Note: in classification groups are known, in clustering they are learned
* Deconvolution -- splitting mixed signals such as instruments or speakers in sound signal

Applications
* Data summarization/compression
* Denoising, outlier detection
* Feature construction

# Clustering

We will first look at a very simple -- and popular -- clustering algorithm called **K-means**

* Randomly choose K cluster center locations (means or centroids)
* Loop until convergence
    1. Assignment of samples to the closest of the K centroids
    2. Re-estimate the cluster	centroids or means based on	the	data assigned to each cluster	

# K-means Observations

1. Initialization is random -- means of the clusters are slightly preturbed versions of data mean
2. Clusters are assumed to be spherical
3. Must manually choose K
3. Clusters can have zero members -- make sure there is no division with zero 

```
mus = numpy.dot(xs,ph.transpose())/(1e-5 + numpy.sum(ph,axis=1))
```
4. There are local minima
   * Non-trivial: poor initialization, too small or too large K
5. Multiple restarts -- from different initializations -- can lead to better solutions


# How do we Come up with an Algorithm such as K-means?

We will:

1. Write out a generative model for the data
2. Write out log-likelihood
3. Optimize log-likelihood

The twist compared to our previous model learning is in the fact that not all the variables are observed

Labels, indicating cluster membership, are **hidden** from us

# Mixture Models -- Generative Story

Each sample $\xx$ is generated by:
1. Selecting a cluster, according to some distribution $\pi = (\pi_1,...,\pi_K)$
$$
p(h) = \pi_h
$$
2. Using parameters of cluster $h$ generate the sample $\xx$
$$
p(\xx \mid h,\theta) = p(\xx \mid \theta_h)
$$

For example in **K-means**, you can think of $\theta_h$ as the mean of the cluster $h$

$$
p(\xx \mid \theta_h) = \prod_{j=1}^p \frac{1}{\sqrt{2\pi\sigma^2}} \myexp{-\frac{1}{2\sigma^2}(x_j - \theta_{h,j})^2}
$$

# Mixture Models -- Generative Story

Note that generative model talks about how data might have been generated

We want to fit a model under which that data is most probable

To do so, we must write out log-likelihood 
$$
\loglik(\Theta) = \sum_i \log p(\xx_i \mid\Theta)
$$
and maximize it with respect to parameters $\Theta$


# Marginal Log-Likelihood 

For a single sample $\xx$, we know how to compute $p(h)$ and $p(\xx\mid h,\Theta)$ so we can obtain joint 

$$
p(\xx,h \mid \Theta) = p(h) p(\xx\mid h,\Theta)
$$

This is probability of the full configuration $\xx$ **and** cluster membership $h$

Our data does not contain information about cluster membership, just vectors $\xx$

**<font color='red'> How do we compute $p(\xx \mid \Theta)$?  </font>**

# Marginal Log-likelihood 

We can use the fact that
$$
p(\xx\mid\Theta) = \sum_h p(\xx,h \mid \Theta)  = \sum_h p(h)p(\xx\mid h,\Theta).
$$

**<font color='red'>  What is the interpretation of this sum? </font>**

# Marginal Log-likelihood
Now, we can express log-likelihood in terms of probabilities in our model

$$
\loglik(\Theta) = \sum_i \log p(\xx_i\mid\Theta) = \sum_{i=1}^N \log  \sum_{h_i} p(\xx_i,h_i\mid \Theta).
$$

Observations
1. For each sample $\xx_i$, we have corresponding cluster membership variable $h_i$
2. There is a sum under the log so we cannot push the log to probability terms

# Dealing with Difficult Objectives

Typically, when we run into an objective that is difficult to work with (for example log of sums above), we seek a closely related objective that is easier

Hence, we will not directly optimize the log-likelihood instead we will introduce a lower-bound on the log-likelihood which we can show is tight at the optimum

To accomplish this we a bit of math that you may not be familar with

1. Convex and concave functions
2. Jensen's inequality

# Concave and Convex Functions 

A function $f(x)$ is concave if

$$
f(\lambda x + (1-\lambda)y) \geq \lambda f(x)  + (1-\lambda)f(y)
$$

for any $\lambda \in [0,1]$

Examples of concave functions are log, exp, square, etc.

Similarly, a function $f(x)$ is convex if

$$
f(\lambda x + (1-\lambda)y) \leq \lambda f(x)  + (1-\lambda)f(y)
$$

for any $\lambda \in [0,1]$

The negative of a concave function is convex

# Jensen's Inequality

Jensen's inequality

$$
f(\EE[H]) \geq \EE[f(H)]
$$

where

$$
\EE[H] = \sum_h p(H=h)h.
$$ 

and $f$ is concave

# Bounding log-Likelihood

Starting with log-likelihood
$$
\loglik(\Theta) = \sum_i \log p(\xx_i\mid\Theta) = \sum_{i=1}^N \log  \sum_{h_i} p(\xx_i,h_i\mid \Theta).
$$
we introduce distributions $q_i(h_i)$ and multiply and divide by them
$$
\begin{aligned}
\loglik(\Theta) &= \sum_i \log p(\xx_i\mid\Theta) = \sum_{i=1}^N \log  \sum_{h_i} \color{red}{\frac{q_i(h_i)}{q_i(h_i)}} p(\xx_i,h_i\mid \Theta)\\
&= \sum_{i=1}^N \log  \sum_{h_i} q_t(h_i) \frac{ p(\xx_i,h_i\mid \Theta) }{ q_i(h_i) } \\
&= \sum_{i=1}^N \log \EE_{q_i}\left[\frac{ p(\xx_i,h_i\mid \Theta) }{ q_i(h_i) }\right] \\
&\geq \sum_{i=1}^N \EE_{q_i}\left[\log \frac{ p(\xx_i,h_i\mid \Theta) }{ q_i(h_i) }\right] \\
&= \sum_{i=1}^N \sum_{h_i} q_i(h_i) \log \frac{ p(\xx_i,h_i\mid \Theta) }{ q_i(h_i) } 
\end{aligned}
$$

# Bounding log-Likelihood

Starting with log-likelihood
$$
\loglik(\Theta) = \sum_i \log p(\xx_i\mid\Theta) \geq  \sum_{i=1}^N \sum_{h_i} q_i(h_i) \log \frac{ p(\xx_i,h_i\mid \Theta) }{ q_i(h_i) }  = \mathcal{B}(\Theta,q)
$$

Natural questions that arise:
1. Where do we get $q_i$s? 
2. Does optimizing bound result in the same $\Theta$?
$$
\argmax_{\Theta}\loglik(\Theta)\stackrel{?}{=}\argmax_{\Theta}\mathcal{B}(\Theta,q) 
$$


# Bounding log-Likelihood

Natural questions that arise:

*  Where do we get $q_i$s? 
 * A: We use posterior probabilities $p(h_i \mid \xx_i,\Theta)$


*  $\argmax_{\Theta} \loglik(\Theta) \stackrel{?}{=} \argmax_{\Theta} \mathcal{B}(\Theta,q)$? 
 * A: Yes, if we use exact posterior probabilities in place of $q_i$s the two objectives coincide and optimal $\Theta$s are the same

# Expectation-Maximization (EM) Algorithm

Hence we can maximize the bound $\mathcal{B}(\Theta)$ by iterating

1. (E-step) Computing the optimal 
$$
q^{\new}_i = \argmax_{q_i} \mathcal{B}(\Theta^{\old},q) 
$$
2. (M-step) Updating $\Theta$ given current $q_i(h_i)$
$$
\Theta^{\new} = \argmax_{\Theta} \mathcal{B}(\Theta,q^{\new}) 
$$

Recall for a moment the K-means algorithm. It alternated analogous two steps:
1. Assigning each sample to a cluster
2. Cluster center computation based on assignments

# EM Algorithm for Mixture of Gaussians with Spherical Covariance

The model
$$
\begin{aligned}
p(h\mid \alpha) &= \alpha_h \\
p(\xx \mid h,\mu) &= (2\pi)^{-\frac{d}{2}} \myexp{-\frac{1}{2}(\xx - \mu_{h_t})^T(\xx - \mu_{h_t})} \\
\end{aligned}
$$
is a variant of **Mixture of Gaussians**

$\alpha_c$ is an a-priori probability that a sample comes from class $c$ -- also called **mixing proportion**

The bound 
$$
\begin{aligned}
\Bound(\Theta,q) &= \sum_{i=1}^N \sum_{h_i} q_i(h_i) \log \frac{ p(\xx_i,h_i\mid \Theta) }{ q_i(h_i) } \\
&=  \sum_{i=1}^T \sum_{h_i} q_i(h_i) \left[ \log \alpha_{h_i} -\frac{d}{2} \log (2\pi) -\frac{1}{2}(\xx - \mu_{h_i})^T(\xx - \mu_{h_i}) \right] \\
&- \sum_{i=1}^T \sum_{h_i} q_i(h_i) \log q_i(h_i)
\end{aligned}
$$
In this case $\Theta = (\alpha_1,...,\alpha_K,\mu_1, ...,\mu_K)$


# E-step

The E-step

$$
\begin{aligned}
q_i(h_i = k) &= p(h_i =k \mid \xx_i, \mu)  = \frac{p(\xx_i,h_i = k \mid \mu)}{\underbrace{\sum_c p(\xx_i,h_i=c \mid \mu)}_{\textrm{same for all values of } k}}\\
 &\propto p(\xx_i,h_i = k\mid \mu)\\
        &=  \alpha_{h_i} (2\pi)^{-\frac{d}{2}} \myexp{-\frac{1}{2}(\xx - \mu_h)^T(\xx - \mu_h)}
\end{aligned}
$$ 


# E-step: Working in Log-Domain

If the data vectors are long, the computation of joint probabilities can yield very tiny probabilities

Rather than working with probabilities we work with log-probabilities. Hence we store 

$$
\log q_i(h_i = k) = \log p(\xx_i,h_i = k \mid \mu) - \log \sum_c p(\xx_i,h_i=c \mid \mu)
$$

If all the probabilities are stored in log-domain, then computation of their sum requires exponentation

$$
\log \sum_c p(\xx_i,h_i=c \mid \mu) = \log \sum_c\exp \ \underbrace{\log p(\xx_i,h_i=c \mid \mu)}_{\textrm{stored log-probability}}
$$

# M-step

In M-step we optimize $\Theta$ given $q$s
$$
\Theta^{\new} = \argmax_{\Theta} \mathcal{B}(\Theta,q^{\new}) 
$$

In general, we can take derivatives, equate them to zero, and solve:
$$
\nabla_{\Theta}  \mathcal{B}(\Theta,q^{\new})  = 0 
$$

We can show that in our case, the M-step updates are:

$$
\begin{aligned}
\mu_c^* &= \frac{\sum_i q_i(c) \xx_i}{\sum_i q_i(c)} \\
\alpha^*_c &= \frac{\sum_i q_i(h_i = c)}{N}
\end{aligned}
$$